In [1]:
!pip install --upgrade flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken '2aAh0gs0N904HTHZSkIHm7tUbji_uSF52Avhxfw2axULYP27'

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=3f3e7ed0f18f0c0132b9d7804df9ad994cac76217224ba361ad0b9116b55fa90
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!cp '/content/drive/MyDrive/All Projects/WeatherPrediction/Module 3' -r '/content/'

In [10]:
%cd '/content/Module 3'

/content/Module 3


In [11]:
from flask import Flask, render_template, jsonify,redirect, url_for, request, session
import json
import utils
from pyngrok import ngrok

app = Flask(__name__)
app.config['SECRET_KEY'] = 'the random string'

news_list = utils.get_news()["articles"][:6]

class Post:
    def __init__(self, title, image_url):
        self.title = title
        self.image_url = image_url


posts = []
for i in range(len(news_list)):
    posts.append(Post(news_list[i]["title"], news_list[i]["urlToImage"]))


@app.route('/', methods=['GET', 'POST'])
def weatherDetails():
    #send data in this format after reciving fron the api (converet the api data to this json format and then send to front end)
    data = utils.get_current_weather_data()
    return render_template('Weather.html',data=data)


def get_map(zone):
    maps = {
        "flood Map": utils.get_flood_map(),
        "lake": utils.get_lake_zones(),
        "drainage": utils.get_drinage_lines(),
    }
    return maps.get(zone, "default_map_url_if_zone_not_found")



@app.route('/floodzone', methods=['POST', 'GET'])
def floodzone():
    if request.method == 'POST':
        selected_zone = request.form.get('zone')
        map_image_url = get_map(selected_zone)
        return render_template('FloodZone.html', map=map_image_url)
    else:
        return render_template('FloodZone.html', map=None)


@app.route('/alert', methods=['POST', 'GET'])
def alert():
    return render_template('Alert.html', posts=posts)



@app.route('/shelter', methods=['POST', 'GET'])
def shelter():
    if request.method == 'POST':
        location_data = request.json  # Assuming JSON data is sent
        session['location_data'] = location_data  # Store the data in session
        return redirect(url_for('shelter_map_page'))

    return render_template('Shelter.html')

@app.route('/shelter_map_page', methods=['GET'])
def shelter_map_page():
    location_data = session.get('location_data')  # Retrieve the data from session
    if location_data:
        latitude, longitude = utils.get_lat_long_for_ip()
        map_html = utils.shelter_map(latitude, longitude)
        return render_template('map.html', map=map_html)
    return "No location data found", 404


@app.route('/sos' , methods=['POST','GET'])
def sos():
   if request.method=='POST':
       name = request.form['name']
       situation = request.form['situation']
       contact = request.form['contact']
       utils.send_email(name, situation, contact)

   return render_template('Sos.html')


ngrok_tunnel = ngrok.connect(5000)
print(ngrok_tunnel)

if __name__ == '__main__':
    app.run(port=5000)

http://aaff-34-80-131-61.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 10:34:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 10:34:47] "GET /static/css/Weather.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 10:34:48] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 10:34:51] "GET /shelter HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 10:34:52] "GET /static/css/Shelter.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 10:34:55] "POST /shelter HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 10:34:56] "GET /shelter_map_page HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 10:34:58] "GET /shelter_map_page HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 10:34:59] "GET /static/css/FloodZone.css HTTP/1.1" 200 -
INFO